In [1]:
import pandas as pd
import sys
import os
sys.path.append('/home/greg/GitHub/')
import pyEDIutils.search as edi
import datetime
import psycopg2
from sqlalchemy import create_engine
now = datetime.datetime.now().strftime("%Y%m%d")

In [2]:

df = edi.search_pasta(fields=['packageid','title','pubdate','author',
                                   'begindate','enddate','doi'])
df.head()

https://pasta.lternet.edu/package/search/eml?defType=edismax&q=%2A&fq=scope%3Aknb-lter-jrn&fl=packageid%2Ctitle%2Cpubdate%2Cauthor%2Cbegindate%2Cenddate%2Cdoi&sort=packageid%2Casc&rows=500


,packageid,title,pubdate,authors,begindate,enddate,doi
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...
1,knb-lter-jrn.101.1,Thresholds and alternative states in neotropic...,2021,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",2005-09-01,2018-03-31,doi:10.6073/pasta/e1978467bd09fc4f6e0cd8509b06...
2,knb-lter-jrn.102.1,Carbon exchange responses of rehydrated and in...,2023,"Hoellrich, Mikaela;Pietrasiak, Nicole",2020-01-01,2022-12-31,doi:10.6073/pasta/c6ffd88dc80df1ed1ec32ccdc477...
3,knb-lter-jrn.200021001.5,Data to explore circular manureshed management...,2023,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",2010-01-01,2019-12-31,doi:10.6073/pasta/6025924b89e313f25bddf1912c8a...
4,knb-lter-jrn.200022001.3,Criollo and Crossbred Steer Comparison: Weight...,2022,"McIntosh, Matt M;Cibils, Andres F;Estell, Rick...",2015-12-01,2017-01-31,doi:10.6073/pasta/66ffe320906785bb708c55a61787...


In [30]:
import sqlalchemy as sq
import yaml

# Get the metabase credentials
with open("/home/greg/admin/jrn_metabase_keys.yaml", "r") as stream:
    try:
        mbcred = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Database engine
engine = sq.create_engine("postgresql+psycopg2://{0}:{1}@{2}:{3}/{4}".format(mbcred['user'],
                                                                             mbcred['password'],
                                                                             mbcred['host'],
                                                                             mbcred['port'],
                                                                             mbcred['mbname']))

# connect to the database using the newly created Engine instance
conn = engine.connect()

# run SQL query
mbdf = pd.read_sql_query(sql='SELECT * FROM lter_metabase."DataSet"', con=conn)
mbdf.head()

,DataSetID,Revision,Title,PubDate,Abstract,ShortName,UpdateFrequency,MaintenanceDescription,AbstractType,BoilerplateSetting
0,210011005,105,Annual ground-based photographs taken at 15 ne...,2019-10-09,abstract.210011005.md,None,annually,ongoing,file,jrn-default
1,210086004,105,Leaf litter cover data on 1m x 1m plots from t...,2022-04-11,abstract.210086004.md,None,asNeeded,ongoing - New data are added every 5 years,file,jrn-default
2,210012002,127,Soil volumetric water content calculated from ...,2019-11-08,abstract.210012002.md,None,notPlanned,None,file,jrn-default
3,210013001,62,Raw neutron counts from a soil water content h...,2019-09-30,abstract.210013001.md,None,notPlanned,None,file,jrn-default
4,210013002,71,Soil volumetric water content calculated from ...,2019-10-29,abstract.210013002.md,None,notPlanned,None,file,jrn-default


In [87]:
df['DataSetID'] = df.packageid.str.split('.',2, expand=True).iloc[:,1].astype(int)

/tmp/ipykernel_533617/3790972908.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['DataSetID'] = df.packageid.str.split('.',2, expand=True).iloc[:,1].astype(int)


In [88]:
df = df.merge(mbdf[['DataSetID','MaintenanceDescription']], on='DataSetID', how='left')

In [99]:
pd.isnull(df.MaintenanceDescription)

0       True
1      False
2      False
3      False
4      False
       ...  
370    False
371    False
372    False
373    False
374    False
Name: MaintenanceDescription, Length: 375, dtype: bool

In [100]:
# Find John't met packages and assign "ongoing"
test = (df['packageid'].str.contains('210437') | df['packageid'].str.contains('210548'))
df.loc[test, 'MaintenanceDescription'] = 'ongoing'
# Find "None" Maintenance descriptions and change to ''
test = pd.isnull(df['MaintenanceDescription'])
df.loc[test, 'MaintenanceDescription'] = ''

In [101]:
import numpy as np
# Create a priority column with empty values
df['priority'] = ''
# Find high priority datasets
test = np.logical_and(df['pubdate'].astype(int) < 2022, df['MaintenanceDescription'].str.contains('ongoing'))
df.loc[test, 'priority'] = 'high'
# Find low priority datasets
test = np.logical_or(df['pubdate'].astype(int) > 2021, df['MaintenanceDescription'].str.contains('complete'))
df.loc[test, 'priority'] = 'low'

In [102]:
df.head()

,packageid,title,pubdate,authors,begindate,enddate,doi,DataSetID,MaintenanceDescription,priority
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...,100,,
1,knb-lter-jrn.101.1,Thresholds and alternative states in neotropic...,2021,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",2005-09-01,2018-03-31,doi:10.6073/pasta/e1978467bd09fc4f6e0cd8509b06...,101,ongoing,high
2,knb-lter-jrn.102.1,Carbon exchange responses of rehydrated and in...,2023,"Hoellrich, Mikaela;Pietrasiak, Nicole",2020-01-01,2022-12-31,doi:10.6073/pasta/c6ffd88dc80df1ed1ec32ccdc477...,102,complete,low
3,knb-lter-jrn.200021001.5,Data to explore circular manureshed management...,2023,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",2010-01-01,2019-12-31,doi:10.6073/pasta/6025924b89e313f25bddf1912c8a...,200021001,complete,low
4,knb-lter-jrn.200022001.3,Criollo and Crossbred Steer Comparison: Weight...,2022,"McIntosh, Matt M;Cibils, Andres F;Estell, Rick...",2015-12-01,2017-01-31,doi:10.6073/pasta/66ffe320906785bb708c55a61787...,200022001,complete,low


In [103]:
df.sort_values('pubdate').to_csv('tables_out/jrn_edi_catalog_bypubdate_20230811.csv')